# Casusopdracht - Films

**Klas:** V2B
**Studenten:** 
- Roan Gaasbeek
- Mathijs
- Luc

### Inhoud ###
- [Onderzoeksvragen](1.-Onderzoeksvragen)

In [1]:
import numpy as np
import pandas as pd

# 1. Onderzoeksvragen
De volgende onderzoeksvragen worden behandeld in deze opdracht:

- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op $Facebook$ en $IMDB$?
- ** zelf nog een vraag bedenken met betrekking op een zelf bijgezochte dataset **
- In hoeverre is het mogelijk om logische clusters te vinden o.b.v. onder andere budget en omzet? Denk aan blockbusters (hoog budget en hoge omzet), flops (hoog budget en lage omzet) of cultfilms (laag budget en hoge omzet). Je mag extra features toevoegen. 

# 2. Data understanding

De eesrte stap in deze opdracht is de **data understanding** fase van **CRISP-DM** dit houdt in:
- Data Collection
- Data Exploration & Analysis

#### 2.1 Data Collection
- de meegekregen movie.csv
- de Bechdeltest

In [2]:
# Dit is de casus dataset
dataset = pd.read_csv('movie.csv')
# dit is de gekozen externe dataset
# komt nog

#### 2.2 Data Exploration & Analysis

In [ ]:
# Om een eerste blik van de dataset te krijgen gebruiken we de head functie
dataset.head()

In [ ]:
# dit is de lijst met de features van onze dataset
dataset.columns

### features en targets
hieronder een lijst van de features met meetniveaus die naar onze mening interessant zijn voor dit onderzoek
- director_name (Nominaal)
- actor_1_name  (Nominaal)
- actor_2_name  (Nominaal)
- actor_3_name  (Nominaal)
- cast_facebook_likes   (Discreet)
- budget    (Continu)
- imdb_score    (Continu)
- movie_facebook_likes  (Discreet)

en de targetvariabel is al bepaald en dat is de opbrengst van de film
- gross (Continu)

In [ ]:
# we gebruiken deze functie om een samenvatting van de features te krijgen.
dataset.describe()

In [ ]:
# met de functie isnull() kunnen we kijken hoeveel van de data we missen 
totaalnull = dataset.isnull().sum()
totaalnull

### Analyse van feature- en targetvariabelen

Te onderzoeken featurevariabelen:  
- movie_facebook_likes
- imdb_score
- budget

Te onderzoeken targetvariabele:  
- gross

In [ ]:
# Kolommen selecteren
kolommen = ['movie_facebook_likes', 'imdb_score', 'budget', 'gross']
data_analyse = dataset[kolommen]
data_analyse


In [ ]:
# Centrum- en spreidingsmaten
data_analyse.describe()

### Waarnemingen centrum- en spreidingsmaten

- De kolommen movie_facebook_likes en imdb_score hebben beide 5043 waarden, maar budget en gross hebben minder waarden (4551 en 4159 waarden).  
We zullen hier een oplossing voor moeten vinden, door bijvoorbeeld de rijen met missende waarden te droppen van de dataset.
- Q1 voor movie_facebook_likes is 0, dit betekent dat ten minste 25% van de films 0 facebook likes hebben. Dit komt waarschijnlijk omdat deze films geen facebook-pagina hebben.


In [9]:
# Uitschieters bepalen

def bepaal_uitschieters(kolom):

        # Q1, Q3 en IQR bepalen
        q1 = data_analyse[kolom].quantile(0.25)
        q3 = data_analyse[kolom].quantile(0.75)
        iqr = q3 - q1

        # Limieten uitschieters berekenen
        klein_laag = q1 - 1.5 * iqr
        klein_hoog = q3 + 1.5 * iqr
        extreem_laag = q1 - 3 * iqr
        extreem_hoog = q3 + 3 * iqr

        print(data_analyse[kolom].name)

        # Kleine uitschieters (1.5 - 3 IQR)
        kleine_uitschieters = data_analyse[kolom][((data_analyse[kolom] < klein_laag) & (data_analyse[kolom] > extreem_laag)) | ((data_analyse[kolom] > klein_hoog) & (data_analyse[kolom] < extreem_hoog))]
        print(f"Kleine uitschieters: {kleine_uitschieters.count()}")
    
        # Extreme uitschieters (3+ IQR)
        extreme_uitschieters = data_analyse[kolom][(data_analyse[kolom] < extreem_laag) | (data_analyse[kolom] > extreem_hoog)]
        print(f"Extreme uitschieters: {extreme_uitschieters.count()}")

In [ ]:
bepaal_uitschieters('movie_facebook_likes')

In [ ]:
bepaal_uitschieters('imdb_score')

In [ ]:
bepaal_uitschieters('budget')

In [ ]:
bepaal_uitschieters('gross')

### Waarnemingen
- movie_facebook_likes heeft 930 extreme uitschieters, dat is behoorlijk veel. Zoals eerder benoemd, komt dit waarschijnlijk doordat er veel films zijn zonder facebook pagina, wat geregistreerd wordt als 0 likes.
- imdb_score heeft 0 extreme uitschieters. Dit lijkt ons logisch omdat de imdb score voor een film altijd tussen de 1 en 10 ligt. Er zal dus nooit een waarde van bijvoorbeeld 100 moeten zijn, wat een extreme uitschieter zou zijn.

# 3. Data preparation
De volgende stap in deze is de data preparation fase van $CRISP-DM$ dit houdt in:
- Data Cleaning
- Data Processing

#### 3.1 Data Processing
*uitleg over wat in dit stuk behandeld wordt*


In [ ]:
dataset.drop(columns=['color', 'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'duration', 'genres', 'num_voted_users', 'facenumber_in_poster','plot_keywords', 'num_user_for_reviews','num_critic_for_reviews', 'language', 'country', 'content_rating', 'title_year','aspect_ratio'], inplace=True)

# we hebben de link voor de imdb pagina niet nodig maar hierin staat wel de imdb_id die we later kunnen gebruiken om een externe dataset te koppelen.
dataset['movie_imdb_link'] = dataset['movie_imdb_link'].str.slice(start=28, stop=-17)

dataset.rename(columns={'director_name':'director', 'movie_title':'title', 'movie_imdb_link':'imdb_id'}, inplace=True)

dataset.head()

*zullen we hier ook nog even de volgorde van kolommen aanpassen om het wat leesbaarder te maken*

#### 3.2 Data Cleaning



In [ ]:
dataset.describe(include="all")
totaal_films = dataset['color'].count()

kleur_films = dataset[dataset['color'] == 'Color'].shape[0]

zw_films = dataset[dataset['color'] != 'Color'].shape[0]  

percentage_kleur = (kleur_films / totaal_films) * 100
percentage_zw = (zw_films / totaal_films) * 100

print(f"Percentage gekleurde films: {percentage_kleur:.2f}%")
print(f"Percentage zwart-wit films: {percentage_zw:.2f}%")


# Color Feature

In [ ]:
#Zoals hierboven zijn bijna 96% van de films in kleur, daarom ervoor gekozen om deze kolom te droppen.
dataset.drop(['color'], axis = 1)


#  Aspect Ratio Feature

In [ ]:
#Aspect ratio is niet relevant
dataset.drop(['aspect_ratio'], axis = 1)


#  Movie Imdb Link Feature

In [ ]:
#Movie Imdb Link is ook niet relevant
dataset.drop(['movie_imdb_link'], axis = 1)


#  Gross Target

In [ ]:
#Hier wordt het gemiddelde van de gross waardes die er zijn berekend
gemiddelde_gross = dataset['gross'].mean()
print(f"Gemiddelde gross: ${gemiddelde_gross:.2f}")


In [ ]:

dataset.fillna({"gross" :48468407.53 })

#  Budget Target

In [ ]:
gemiddelde_budget = dataset['budget'].mean()
print(f"Gemiddelde budget: ${gemiddelde_budget:.2f}")


In [ ]:
dataset.fillna({"budget" :48468407.53 })

# Datatypes